In [ ]:
import pandas as pd
import numpy as np
import random as python_random

# The below is necessary for starting Numpy generated random numbers
np.random.seed(13)
# The below is necessary for starting core Python generated random numbers
python_random.seed(13)

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow import random as tf_random

# The below set_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
tf_random.set_seed(13)

In [ ]:
import os
import librosa
import librosa.display
import glob 
import skimage

In [ ]:
df = pd.read_csv("UrbanSound8K.csv")

df.head(10)

In [ ]:
dat1, sampling_rate1 = librosa.load('./fold5/100032-3-0-0.wav')
arr = librosa.feature.melspectrogram(y=dat1, sr=sampling_rate1)
arr.shape

In [ ]:
feature = []
label = []

# Veri setini işlenebilir hale getiren fonksiyon
def parser(row):
    for i in range(df.shape[0]):
        file_name = './fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
        # Hızlı yüklemek için kaiser_fast kullanıldı.
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        # MFCC Feature. Sütunların aritmetik ortalaması ile flatten array oluşturuldu.
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)        
        feature.append(mels)
        label.append(df["classID"][i])

    print(str(df.shape[0]) + " extraction completed.")
    return [feature, label]

In [ ]:
temp = parser(df)

In [ ]:
temp = np.array(temp)
data = temp.transpose()

In [ ]:
print(len(data[:, 0][0]))

In [ ]:
X_ = data[:, 0]
Y = data[:, 1]
print(X_.shape, Y.shape)
X = np.empty([8732, len(data[:, 0][0])])

In [ ]:
for i in range(X_.shape[0]):
    X[i] = (X_[i])

In [ ]:
Y = to_categorical(Y)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
#Dataseti train ve test olmak üzere 2'ye ayırıyor. Random_state seed atayarak
#her seferinde aynı şekilde bölmeyi sağlıyor. Stratify'da her çıktı class'ını
#oranlayarak bölmeyi sağlıyor.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 13, stratify=Y)

In [ ]:
#Input_dim
num_rows = 16
num_columns = 8
num_channels = 1

#128 boyutunda olan veriyi CNN kullanmak amacı ile 16,8,1 boyutlarında reshape yapıyor.
X_train = X_train.reshape(6549, num_rows, num_columns, num_channels)
X_test = X_test.reshape(2183, num_rows, num_columns, num_channels)

In [ ]:
num_rows = X_train.shape[1]
num_columns = X_train.shape[2]
num_channels = X_train.shape[3]
input_dim = (num_rows, num_columns, num_channels)

In [ ]:
#Model oluştuluyor.
model = Sequential()

In [ ]:
#Layerlar ekleniyor.
model.add(Conv2D(64, (3, 3), padding = "same", activation = "relu", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, epochs = 100, batch_size = 128, validation_data = (X_test, Y_test))

In [ ]:
model.summary()

In [ ]:
#accuracy graph
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#loss graph
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#Accuracy
predictions = model.predict(X_test)
score_train = model.evaluate(X_train, Y_train)
score_test = model.evaluate(X_test, Y_test)
print(score_train)
print(score_test)

In [ ]:
#Confusion Matrix
preds = np.argmax(predictions, axis = 1)
cm = confusion_matrix(Y_test.argmax(axis=1), preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, xticklabels=["air_conditioner","car_horn","children_playing","dog_bark","drilling","engine_idling","gun_shot","jackhammer","siren","street_music"], yticklabels=["air_conditioner","car_horn","children_playing","dog_bark","drilling","engine_idling","gun_shot","jackhammer","siren","street_music"], annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()